In [1]:
from konlpy.tag import Twitter
from gensim.models import Word2Vec
import csv
from konlpy.tag import Twitter
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Okt
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

C:\Users\ICT01_21\Anaconda3\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ICT01_21\Anaconda3\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ICT01_21\Anaconda3\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ICT01_21\Anaconda3\envs\tf_test\lib\site-packages

In [3]:
data_set = pd.read_excel("data.xlsx")
data_set = data_set.iloc[:,1:]
data_set.rename({0:'news'},axis=1, inplace= True)
data_label = {1:'culture',2:'global',3:'politic',4:'society',5:'economy'}




In [5]:
data_set.head()

,news,label
0,[날씨] 단비에 건조특보 해제...남은 연휴 맑지만 中 스모그,1
1,[날씨] 설 아침 춥지만 낮 포근...새벽 영서 북부 눈발,1
2,"[카드뉴스] '올해로 34년째, 열차 안에서 보내는 설날'",1
3,"[인터뷰] 김영민 교수 ""조카가 '당숙이란 무엇인가' 묻는다면...""",1
4,[날씨] 다소 쌀쌀한 ‘입춘’…수도권 미세먼지 ‘나쁨’,1


In [6]:
for i in range(len(data_set)) :
    data_set['news'][i] = re.sub('[^a-zA-Z0-9 ㄱ-ㅣ가-힣]', '', data_set['news'][i])
    


C:\Users\ICT01_21\Anaconda3\envs\tf_test\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
data_set = data_set.sample(frac=1) # frac 전체 row에서 몇%의 데이터를 return할 것인지 정할 수 있다.

In [8]:
data_set.head()

,news,label
27178,청와대 떠나는 조국 소임 마쳐 영광야유 보낸 언론도 존중,3
33042,필리핀서 돌아온 폐기물 재활용 불가처리 놓곤 핑퐁,4
44757,지난해 1인당 국민소득 3만1000달러 돌파체감은 글쎄,5
20946,난폭한 신도에 화낸 교황인내심 잃어 사과,2
28137,독도훈련까지 번지는 지소미아 한일 악화 의도로 본 듯,3


In [9]:
def get_noun(text):
    tokenizer = Twitter()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

def get_noun2(text):
    tokenizer = Hannanum()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

def get_noun3(text):
    tokenizer = Okt()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

def get_morphs3(text):
    tokenizer = Okt()
    morphs = tokenizer.morphs(text)
    return [n for n in morphs]

def get_noun4(text):
    tokenizer = Komoran()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]


In [10]:

CV = CountVectorizer(tokenizer = get_noun3)
CV2 = CountVectorizer(tokenizer = get_morphs3)


In [11]:
CV = CountVectorizer(tokenizer = get_noun3)

In [12]:
train_set, test_set = train_test_split(data_set, test_size = 0.3, random_state = 42)


In [13]:
dic = { 
        '기자':'가',
        '연합뉴스': '가',
        '뉴스':'가',
        '사진': '가',
        '무단': '가',
        '금지':'가',
        '영상': '가',
        '동아일보':'가',
        '제공':'가',
        '전문':'가',
        '종합':'가',
        '단독':'가',
        '속보':'가',
        '이슈':'가',
        '속':'가'}

In [14]:
data_set = data_set.replace(dic,'')

In [15]:
fit_transform = CV.fit_transform(train_set["news"])

In [16]:
data_set

,news,label
27178,청와대 떠나는 조국 소임 마쳐 영광야유 보낸 언론도 존중,3
33042,필리핀서 돌아온 폐기물 재활용 불가처리 놓곤 핑퐁,4
44757,지난해 1인당 국민소득 3만1000달러 돌파체감은 글쎄,5
20946,난폭한 신도에 화낸 교황인내심 잃어 사과,2
28137,독도훈련까지 번지는 지소미아 한일 악화 의도로 본 듯,3
...,...,...
18856,영국 경찰 냉동 컨테이너 시신 39구 신원 확인 시간 걸릴 듯,2
18971,독신주의 900년 만에아마존서 결혼한 사제 나오나,2
46740,단독 국세청 행사 두 번 펑크 낸 배우 서현진,5
38280,공식입장강타우주안 입맞춤 영상 돌연 삭제실수로 올,4


In [95]:
from konlpy.tag import Twitter
c = []
for i in range(len(data_set["news"])):
    twitter = Twitter()
    c.append(twitter.nouns(list(data_set["news"])[i]))

In [96]:
c

[['청와대', '조국', '소임', '영광', '야유', '언론', '존중'],
 ['필리핀', '폐기물', '재활용', '불가', '처리', '핑퐁'],
 ['지난해', '국민소득', '돌파', '체감', '글쎄'],
 ['신도', '화', '교황', '인내심', '사과'],
 ['독도', '훈련', '번지', '미아', '한일', '악화', '의도', '듯'],
 ['나', '자연인', '출연자', '미성년', '내', '딸', '성추행'],
 ['옥살이', '차', '화성', '범인', '줄곧', '고문', '허위', '자백', '주장'],
 ['단독', '칠레', '방치', '한국', '땅', '태양광', '발전소'],
 ['선임', '기자', '경제', '노트', '중산층', '서울', '새', '아파트', '갈증'],
 ['줄줄', '보조금', '가짜', '농부', '행세', '억대', '타', '공무원'],
 ['유정', '팥', '소금', '김상중', '알', '최고', '충격'],
 ['엑소', '팬', '첸', '퇴', '이기', '선택'],
 ['봄', '성큼', '미세먼지', '시민'],
 ['어머니', '건강', '안', '윤지오', '돌연', '출국', '귀국', '질문'],
 ['확', '진자', '딸', '부부', '신종', '코로나', '음성'],
 ['당', '이미선', '거듭', '사퇴', '요구', '주식', '팔', '재판', '관직', '사려', '하나'],
 ['출연', '혐한', '발언', '어간', '변호사', '한국', '사장', '왜', '멋'],
 ['영상', '무장해제', '일본군', '눈빛', '광복', '정부', '수'],
 ['단독', '마약', '현대', '세', '텔레그램', '정회장'],
 ['여자친구', '클럽', '집단', '폭행', '사망'],
 ['윤지오', '등', '후원자', '결정', '거짓', '증언', '작품', '표절', '의혹'],
 ['토모', '나라', '한국', '나라',

In [105]:
from konlpy.tag import Twitter
c = []
for i in range(len(data_set["news"])):
    twitter = Twitter()
    c.append(twitter.nouns(data_set["news"][i]))

In [106]:
c

[['날씨', '단비', '건조', '특보', '해제', '연휴', '스모그'],
 ['날씨', '설', '아침', '낮', '포', '새벽', '영서', '북부', '눈발'],
 ['카드', '뉴스', '올해', '열차', '안', '설날'],
 ['인터뷰', '김영민', '교수', '조카', '당숙', '무엇'],
 ['날씨', '다소', '입춘', '수도권', '미세먼지'],
 ['날씨', '절기', '입춘', '황사', '중국', '발', '스모그', '미세먼지'],
 ['날씨', '설날', '미세먼지', '일시', '아침', '영하', '추위'],
 ['날씨', '미세먼지', '입춘', '낮', '서울', '도'],
 ['인간', '돌고래', '숭어', '사냥'],
 ['날씨', '미세먼지', '입춘', '한낮', '서울', '도'],
 ['결혼', '안', '더', '명절', '도망'],
 ['킹덤', '배두나', '연기력', '논란', '예상', '신분', '사극', '톤', '포기'],
 ['페미니스트', '결혼', '하나요', '장손', '남편', '호박', '전'],
 ['디지털', '스토리', '차례상', '생략', '전통'],
 ['사연', '뉴스', '아내', '오빠', '아내', '아주머니', '정말'],
 ['살', '안', '체질', '정말', '유전자', '연구', '준'],
 ['가족', '발견', '가족', '극단', '선택', '혼자', '희망이'],
 ['좌우', '습관'],
 ['스트레스', '때', '몸', '신호'],
 ['승리', '이름', '진짜', '과거', '발언', '조명'],
 ['동물병원', '폐업', '안락사', '위기', '구조', '몰티즈'],
 ['김지영', '일본', '여성', '독자', '한국', '문학'],
 ['뱃살', '음식'],
 ['범인', '잡고', '닭', '잡고', '배꼽', '잡고', '극장', '코믹', '영화', '열풍'],
 ['동남아', '여행', '홍역', '조심', '

In [97]:
import numpy as np
from sklearn.model_selection import train_test_split
train, test = train_test_split(c, test_size = 0.33, random_state = 42)


In [109]:
from konlpy.tag import Komoran
 
nlp = Komoran()

noun_list = []
for i in range(len(data_set["news"])):
    twitter = Twitter()
    noun_list.append(twitter.nouns(data_set["news"][i]))
result = {}
# 단어 검색 및 추가
for noun in noun_list:
    # 처음 찾아진 단어일 경우
    if result.get(noun) == None:
        result[noun] = 1
    # 이미 있는 단어일 경우
    else:
        result[noun] += 1
 
# 빈도 계산해서 출력
for noun in result:
    print(noun + " : " + str(result[noun]) + "회(" + str(round(result[noun]/len(result), 2)) + "%)")


TypeError: unhashable type: 'list'

In [88]:
from konlpy.tag import Kkma
kkma = Kkma()

kkma.nouns(data_set["news"][1])

['날씨', '설', '아침', '낮', '포', '포근새벽', '근', '새벽', '영서', '북부', '눈발']

In [32]:
CV.vocabulary_

{'이경규': 13721,
 '마장면': 5366,
 '출시': 17592,
 '가격': 2,
 '후기': 20887,
 '트럼프': 18704,
 '화웨이': 20632,
 '전면': 15190,
 '금지': 2193,
 '국가': 1758,
 '비상': 8303,
 '사태': 8728,
 '선포': 9349,
 '갈등': 238,
 '격랑': 753,
 '속': 9780,
 '알바니아': 11380,
 '최악': 17427,
 '지진': 16456,
 '발생': 6911,
 '최소': 17417,
 '명': 5792,
 '사망': 8582,
 '존스홉킨스대': 15883,
 '신종': 10786,
 '코로나': 18022,
 '실시간': 10862,
 '확산': 20670,
 '지도': 16354,
 '공개': 1253,
 '고민정': 1077,
 '아수라': 11138,
 '걱정': 625,
 '대한민국': 4120,
 '터키': 18431,
 '쿠르드': 18124,
 '군사작전': 1893,
 '임박': 14388,
 '대통령실': 4094,
 '곧': 1198,
 '시리아': 10534,
 '국경': 1773,
 '종': 15907,
 '서울': 9202,
 '상': 8896,
 '하위': 19674,
 '차': 16792,
 '최대': 17391,
 '무인기': 6178,
 '격추': 767,
 '보복': 7591,
 '공격': 1255,
 '승인': 10474,
 '철회': 17128,
 '속보': 9790,
 '북한': 7982,
 '단거리': 3727,
 '미사일': 6431,
 '신뢰': 10710,
 '위반': 13193,
 '생각': 9075,
 '안해': 11359,
 '친박': 17753,
 '핵심': 20099,
 '이정현': 14013,
 '은혜': 13581,
 '보답': 7553,
 '위해': 13240,
 '순천': 10211,
 '내년': 3171,
 '총선': 17363,
 '수도권': 9989,
 '어산지': 11731

In [30]:
word_test = CV.vocabulary_

In [20]:
# word_test = np.asarray(word_test)

In [46]:
list(word_test.values())[2]

21057

In [32]:
a = []
for i in range(len(word_test)):
    a.append([list(word_test.keys())[i],list(word_test.values())[i]])

In [33]:
a

[['세계', 9580],
 ['위', 13175],
 ['희소식', 21057],
 ['잠시', 14749],
 ['조선', 15775],
 ['드릴', 4700],
 ['악재', 11299],
 ['적자', 15116],
 ['쇼크', 9988],
 ['상주', 9047],
 ['고속', 1098],
 ['블랙', 8255],
 ['아이스', 11207],
 ['사고', 8575],
 ['명', 5829],
 ['사망', 8626],
 ['부상', 7879],
 ['부산', 7876],
 ['일본', 14311],
 ['뱃길', 7269],
 ['승객', 10481],
 ['급감', 2216],
 ['대마도', 3998],
 ['행', 20110],
 ['하루', 19629],
 ['편', 19102],
 ['방', 7020],
 ['잡기', 14762],
 ['하늘', 19613],
 ['별', 7517],
 ['요강', 12787],
 ['여관', 12005],
 ['탑승', 18337],
 ['못', 6085],
 ['해', 19983],
 ['탑승객', 18338],
 ['전원', 15255],
 ['에티오피아', 11952],
 ['여객기', 11999],
 ['애플', 11538],
 ['이모티콘', 13808],
 ['안드로이드', 11338],
 ['카톡', 17849],
 ['집결', 16642],
 ['예고', 12380],
 ['유총', 13430],
 ['교육부', 1612],
 ['부처', 7951],
 ['합심', 19924],
 ['강경', 348],
 ['대응', 4069],
 ['김정은', 2645],
 ['그', 2048],
 ['도보', 4311],
 ['다리', 3668],
 ['린다', 5283],
 ['침묵', 17779],
 ['반쪽', 6902],
 ['개방', 497],
 ['윤지오', 13541],
 ['거짓', 632],
 ['증언', 16304],
 ['논란', 3448],
 ['안민석', 11345],
 

In [121]:
import numpy as np
from sklearn.model_selection import train_test_split
train, test = train_test_split(a, test_size = 0.33, random_state = 42) # 테스트셋 3분의 1
train
# model = Word2Vec(train ,window = 5,min_count=2,sg=1,iter=100) # window크기 5, 최소 출현수 2, skip-gram, 10000번 학습

[['암', 11437],
 ['이목', 13809],
 ['흑백', 21003],
 ['배경', 7125],
 ['구강', 1654],
 ['대기발령', 3958],
 ['희토류', 21059],
 ['온에어', 12584],
 ['서식', 9245],
 ['소모', 9738],
 ['도호', 4378],
 ['토트넘', 18527],
 ['먹이', 5711],
 ['날고기', 3055],
 ['야유회', 11604],
 ['힐', 21080],
 ['공정위', 1355],
 ['종적', 15951],
 ['송선미', 9942],
 ['즐비', 16282],
 ['빈속', 8457],
 ['불티', 8188],
 ['회의', 20839],
 ['호칭', 20457],
 ['바치', 6703],
 ['임한솔', 14437],
 ['푸대접', 19356],
 ['소망', 9730],
 ['자세', 14595],
 ['맛', 5556],
 ['스벅', 10340],
 ['집사', 16662],
 ['구축', 1751],
 ['성기구', 9471],
 ['즉흥', 16280],
 ['빼', 8522],
 ['백제', 7248],
 ['문병호', 6292],
 ['마운트', 5371],
 ['윤창중', 13544],
 ['울음소리', 13016],
 ['수술복', 10105],
 ['강성태', 396],
 ['나시', 2937],
 ['미인대회', 6517],
 ['저스틴', 15064],
 ['슈퍼스타', 10310],
 ['이석기', 13885],
 ['창녕군', 16934],
 ['경복궁', 885],
 ['두산위브더제니스', 4631],
 ['관대', 1465],
 ['제제', 15686],
 ['비디오', 8316],
 ['개덜', 481],
 ['시식', 10602],
 ['나고야시', 2902],
 ['봉건', 7792],
 ['관광객', 1459],
 ['노부', 3375],
 ['직분', 16501],
 ['또봉', 4849],
 ['최장집', 174

In [48]:
print('한국 : ', model['이경규'])

C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


KeyError: "word '이경규' not in vocabulary"

In [116]:
from konlpy.tag import Komoran
 
nlp = Komoran()
noun_list = {}
for i in range(len(data_set["news"])) :
    noun_list.append(nlp.nouns(data_set["news"][i]))
 
result = {}
# 단어 검색 및 추가
for noun in noun_list:
    # 처음 찾아진 단어일 경우
    if result.get(noun) == None:
        result[noun] = 1
    # 이미 있는 단어일 경우
    else:
        result[noun] += 1
 
# 빈도 계산해서 출력
for noun in result:
    print(noun + " : " + str(result[noun]) + "회(" + str(round(result[noun]/len(result), 2)) + "%)")



java.lang.OutOfMemoryError: Java heap space

list

In [ ]:


model = Word2Vec(data_set['news'], min_count=3, iter=20, size=300,sg=1,
                window = 5)
model.init_sims(replace=True)